In [12]:
###########################
### MODULES NECESSAIRES ###
###########################


# Import the necessary libraries
import numpy as np
import pandas as pd
import math
import fonctionsSupervisedLearning2 as fsl2
import seaborn as sns

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

from auxFonctions import AminoAcid

In [6]:
# Read data from a file into a list of entries
with open('data/SIG_13.red', 'r') as file:
    entries = file.read().split('\n   ')

In [7]:
##############################
## RECUPERATION DES DONNÉES ##
##############################

# Process each entry
processed_entries = [fsl2.process_entry(entry) for entry in entries]

# Create a DataFrame
df = pd.DataFrame(processed_entries)

# Get the position of the cleavage site
cleavage_site_position = df['Annotation'].apply(lambda x: x.find('C'))

# Split the primary structure into a list of amino acids
amino_acid_seq = df['Primary Structure'].apply(lambda x: list(x))

## Traitement des données

Put words in vector and vice versa

In [8]:
df_exploitable = fsl2.convert_df_to_vectors2(df)

In [21]:
n = 15
p,q = 13, 2  # p+q = 15

In [22]:
# Create a DataFrame to store, for each primary structure, the neihborhood of the cleavage site
# The neighborhood is defined as the word of length p+q starting p letters before the cleavage site
correct_neighborhood = pd.Series()
for i, seq in amino_acid_seq.items():
    correct_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p:cleavage_site_position[i]+q])

In [23]:
# Create a DataFrame to store, for each primary structure, a sequence that is not the neighborhood of the cleavage site
incorrect_neighborhood = pd.Series()
decalage  = [1,2,3,4,5, -1,-2,-3,-4, -5]
for i, seq in amino_acid_seq.items():
    dec = np.random.choice(decalage)
    dec = 0 if cleavage_site_position[i]-13 - dec < 0 else dec
    incorrect_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p-dec:cleavage_site_position[i]+q-dec])

In [24]:
#print(correct_neighborhood.get(4))
print(fsl2.word_to_vector(incorrect_neighborhood.get(52)))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

## Train the models

In [6]:
X_train, X_test, bool_train, bool_test = fsl2.test_train_split_random_pos2(df_exploitable, 15, 13, random_state=42)

count_true = 0
count_all = 0
for t in bool_train:
    count_all += 1
    if t:
        count_true += 1

print(count_true)
print(count_all)
print(bool_train[0])

TypeError: extract_random_subsequence2() missing 1 required positional argument: 'row'

In [11]:
#svm_model_nei, accuracy_nei = fsl2.create_model2(15, df_exploitable, random_state=42, nb_letters = 26, kernel_neighbor= 'rbf',C_nei = 1)


C_list = [0.1,1,2, 5]
kernel_list = ['linear', 'rbf', 'sigmoid', 'poly']
p_list = [3, 5, 7, 9, 11, 13, 15]
q_list = [2, 4, 6, 8]
results = []

for C_nei in C_list:
    for kernel_nei in kernel_list:
        for p in p_list:
            for q in q_list:
                n = p+q
                correct_neighborhood = pd.Series()
                for i, seq in amino_acid_seq.items():   
                    correct_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p:cleavage_site_position[i]+q])

                incorrect_neighborhood = pd.Series()
                decalage  = [1,2,3,4,5, -1,-2,-3,-4, -5]
                for i, seq in amino_acid_seq.items():
                    dec = np.random.choice(decalage)
                    dec = 0 if cleavage_site_position[i]-p - dec < 0 else dec
                    incorrect_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p-dec:cleavage_site_position[i]+q-dec])

                svm_model_nei, accuracy_nei = fsl2.create_model2(n, p, df_exploitable, kernel_nei, C_nei)

                count_fn = 0
                for i in range(0, len(correct_neighborhood)):
                    x = np.array([fsl2.word_to_vector(correct_neighborhood.get(i))])
                    test = svm_model_nei.predict(x)
                    if (test == 0):
                        count_fn += 1
                count_fp = 0
                for i in range(0, len(incorrect_neighborhood)):
                    x = np.array([fsl2.word_to_vector(incorrect_neighborhood.get(i))])
                    test = svm_model_nei.predict(x)
                    if (test == 1):
                        count_fp += 1
            
                results.append((C_nei, kernel_nei, p, q, accuracy_nei, count_fn/len(correct_neighborhood), count_fp/len(incorrect_neighborhood)))

results_df = pd.DataFrame(results, columns=['C','Kernel', 'p', 'q', 'Accuracy', 'False negative rate', 'False positive rate'])
results_df.to_csv('results2.csv', index=True)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1408,) + inhomogeneous part.

In [4]:
results_df = pd.read_csv('results2.csv', index_col=0)

best_results = results_df.loc[(results_df['Accuracy']).idxmax()]
print(best_results)

C                           0.1
Kernel                  sigmoid
p                             9
q                             4
Accuracy               0.897163
False negative rate    0.180398
False positive rate    0.244318
Name: 61, dtype: object


In [ ]:
# Load the results from CSV
results_df = pd.read_csv('results2.csv')

# Setting the aesthetic style of the plots
sns.set(style="whitegrid")

# Create a series of plots
def plot_results(data, x, y, hue, col=None, kind='line', col_wrap=4, height=3):
    """
    A function to create a FacetGrid plot for the specified parameters.
    """
    g = sns.relplot(
        data=data,
        x=x, y=y, hue=hue, 
        col=col, col_wrap=col_wrap,
        kind=kind,
        height=height,
        facet_kws={'sharey': False, 'sharex': False}
    )
    g.set_axis_labels(x, f"{y} Rate")
    g.set_titles("{col_name}")
    g.add_legend()
    plt.show()

# Plot accuracy by C parameter for each kernel type
plot_results(results_df, x='C_nei', y='accuracy_nei', hue='kernel_nei', col='kernel_nei', kind='line')

# Plot false negative rate by C parameter for each kernel type
plot_results(results_df, x='C_nei', y='false negative', hue='kernel_nei', col='kernel_nei', kind='line')

# Plot false positive rate by C parameter for each kernel type
plot_results(results_df, x='C_nei', y='false positive', hue='kernel_nei', col='kernel_nei', kind='line')

# Additional plots can be created for other relationships
# For example, how the accuracy changes with p and q
plot_results(results_df, x='p', y='accuracy_nei', hue='q', col='kernel_nei', kind='line', col_wrap=2)


In [12]:
svm_model_nei, accuracy_nei = fsl2.create_model2(15, df_exploitable,'rbf', 1.0)

In [13]:
print(accuracy_nei)

0.8687943262411347


In [14]:
count = 0
for i in range(0, len(correct_neighborhood)):
    X = fsl2.word_to_vector(df["Primary Structure"].get(i))
    if fsl2.find_cleavage2(X,svm_model_nei,15):
        count += 1

print(count)


1404


In [15]:
count_fn = 0
for i in range(0, len(correct_neighborhood)):
    test = fsl2.is_neighborhood2(correct_neighborhood.get(i), svm_model_nei, 15)
    if not(test):
        count_fn += 1

print("Nombre de faux négatifs :" + str(count_fn))

count_fp = 0
for i in range(0, len(incorrect_neighborhood)):
    try : 
        test = fsl2.is_neighborhood2(incorrect_neighborhood.get(i), svm_model_nei, 15)
        if test:
            count_fp += 1
    except:
        print("Erreur" + str(i))
        #print(incorrect_neighborhood.get(i))


print("Nombre de faux positifs :" + str(count_fp))
print("Nombre total :" + str(len(correct_neighborhood)))




Nombre de faux négatifs :126
Nombre de faux positifs :460
Nombre total :1408
